In [40]:
# Load Seurat object and create pseudobulk for MOFA analysis
# Necessary cell annotations in the seurat object are: 'cluster_id' (giving the cell-type cluster annotations), 'sample_id' (unique identifier of a sample)

#############################################
# Prerequisites - Load Libraries

In [41]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/mofa_analysis//lib/R/library"


In [42]:
source('MS2_Plot_Config.r')

###############################################
# Preqrequisites Configurations & Parameters

In [43]:
### Load the parameters that are set via the configuration files

In [44]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [45]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [46]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [47]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [48]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [49]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [50]:
## Loading the file containing the name of the single-cell dataset

In [51]:
sc_configs = read.csv('configurations/01_Pre_Processing_SC_Data.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/01_Pre_Processing_SC_Data.csv'”


In [52]:
head(sc_configs,2)

,data_name,data_type
,<chr>,<chr>
1,Prepared_sc_Data,h5seurat


In [53]:
sc_configs = sc_configs[sc_configs$data_name != '',]

In [54]:
sc_dataset_names = unique(sc_configs$data_name)

In [55]:
sc_dataset_names

[1] "Prepared_sc_Data"

In [56]:
### Generate the result data directory if it does not exist yet
if(!file.exists(paste0(result_path, '01_results'))){
    dir.create(file.path(paste0(result_path, '01_results')))
    }

# Convert Data

## Load Seurat object

In [57]:
### Load single-cell datasets as h5seurat format; should contain the meta-columns: sample_id; cluster_id
### Raw Counts should be given in default assay

In [58]:
sc_data_list = list()

In [59]:
###### Load the generated seurat objects
for(i in sc_dataset_names){
    source_text = paste( data_path, '/', i , '.h5seurat', sep = '')
    print(source_text)
    print(file.info(source_text)$mtime)
    Convert(source_text, dest = "h5ad")
    #rna_seurat_data = LoadH5Seurat(source_text, assays = "RNA", quietly = TRUE )
    
    #sc_data_list[[i]] = rna_seurat_data
    }


[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data//Prepared_sc_Data.h5seurat"
[1] "2023-12-08 10:27:41 CET"


Validating h5Seurat file

Adding scale.data from RNA as X

Transfering meta.features to var

Adding data from RNA as raw

Transfering meta.features to raw/var

Transfering meta.data to obs

